In [1]:
#!pip install protobuf

In [2]:
#!pip install sklearn_crfsuite eli5

In [61]:
import eli5
import nltk
import scipy.stats
import sklearn
import sklearn_crfsuite

from itertools import chain
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

nltk.download('conll2002')

[nltk_data] Downloading package conll2002 to
[nltk_data]     /Users/bootcamp/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


True

### Загрузим данные:

In [62]:
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

In [63]:
len(train_sents)

8323

### Добавим фичи для каждого слова, чтобы обучить CRF (смотри лекцию:)):

In [73]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    articles = ("a", "de", "con", "por", "para", "en", "ante", "bajo", "contra", "desde", "entre", "hasta", "segun", "sin", "sobre", "tras", "hacia", "detrás")
    sigles = ("uno", "un", "una", "uno")
    pointers = ("yo", "tú", "ella", "Él".lower(), "usted", "ustedes", "vosotros", "vosotras", "nosotros", "nosotras")
    this = ("este", "esto", "esta", "estos", "estas", "eso", "esa", "esos")
    
    features = {
        'bias': 1.0,
        'postag': postag,
        'word.len': len(word),
        ### YOUR CODE HERE
        'word.isupper': word.isupper(),
        'word.istitle': word.istitle(),
        'word.numeric': word.isnumeric(),
        'word.isaplha': word.isalpha(),
        'word.xx': word in ("lo", "la", "le"),
        "word.endswith_punctuation": word in (".", "?", "!"),
        "word.article": word in articles,
        "word.sigle": word in sigles,
        "word.pointer": word in pointers,
        "word.last2letters": word[:-2] if len(word) > 1 else word,
        "word.last_letter": word[:-1] if len(word) > 0 else word
    }
    
    if len(word) >= 2:
        features["word.verb_inf"] = word[-2:] in ("ar", "ir", "er")
    
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.istitle': word1.istitle(),
            ### YOUR CODE HERE
            '-1.postag': postag1,
            '-1.word.len': len(word1),
            '-1.word.isupper': word1.isupper(),
            '-1.word.numeric': word1.isnumeric(),
            '-1.word.isaplha': word1.isalpha(),
            '-1.word.verb_inf': word1[-2:] in ("ar", "ir", "er"),
            '-1.word.xx': word1 in ("lo", "la", "le"),
            "-1.word.endswith_punctuation": word1 in (".", "?", "!"),
            "-1.word.article": word1 in articles,
            "-1.word.sigle": word in sigles,
            "-1.word.pointer": word in pointers,
            "-1.word.last2letters": word[:-2] if len(word) > 1 else word,
            "-1.word.last_letter": word[:-1] if len(word) > 0 else word
        })
        if len(word1) >= 2:
            features["-1.word.verb_inf"] = word1[-2:] in ("ar", "ir", "er")
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.istitle': word1.istitle(),
            ### YOUR CODE HERE
            '+1.postag': postag1,
            '+1.word.len': len(word1),
            '+1.word.isupper': word1.isupper(),
            '+1.word.numeric': word1.isnumeric(),
            '+1.word.isaplha': word1.isalpha(),
            '+1.word.verb_inf': word1[-2:] in ("ar", "ir", "er"),
            '+1.word.xx': word1 in ("lo", "la", "le"),
            "+1.word.endswith_punctuation": word1 in (".", "?", "!"),
            "+1.word.article": word1 in articles,
            "+1.word.sigle": word in sigles,
            "+1.word.pointer": word in pointers,
            "+1.word.last2letters": word[:-2] if len(word) > 1 else word,
            "+1.word.last_letter": word[:-1] if len(word) > 0 else word
        })
        if len(word1) >= 2:
            features["+1.word.verb_inf"] = word1[-2:] in ("ar", "ir", "er")
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

### Посмотрим на пример фичей для одного слова:

In [74]:
X_train[0][1]

{'bias': 1.0,
 'postag': 'Fpa',
 'word.len': 1,
 'word.isupper': False,
 'word.istitle': False,
 'word.numeric': False,
 'word.isaplha': False,
 'word.xx': False,
 'word.endswith_punctuation': False,
 'word.article': False,
 'word.sigle': False,
 'word.pointer': False,
 'word.last2letters': '(',
 'word.last_letter': '',
 '-1:word.istitle': True,
 '-1.postag': 'NP',
 '-1.word.len': 9,
 '-1.word.isupper': False,
 '-1.word.numeric': False,
 '-1.word.isaplha': True,
 '-1.word.verb_inf': False,
 '-1.word.xx': False,
 '-1.word.endswith_punctuation': False,
 '-1.word.article': False,
 '-1.word.sigle': False,
 '-1.word.pointer': False,
 '-1.word.last2letters': '(',
 '-1.word.last_letter': '',
 '+1:word.istitle': True,
 '+1.postag': 'NP',
 '+1.word.len': 9,
 '+1.word.isupper': False,
 '+1.word.numeric': False,
 '+1.word.isaplha': True,
 '+1.word.verb_inf': False,
 '+1.word.xx': False,
 '+1.word.endswith_punctuation': False,
 '+1.word.article': False,
 '+1.word.sigle': False,
 '+1.word.pointer':

### Обучим CRF:

In [66]:
%%time
### YOUR CODE HERE (Probably you will change some hyperparameters)
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.5,
    c2=0.5,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 41.1 s, sys: 1.4 s, total: 42.5 s
Wall time: 44.5 s


### Посмотрим на веса признаков:

In [78]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.5,
    c2=0.5,
    max_iterations=150,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)
eli5.show_weights(crf, top=30)

### Посчитаем предсказание на тесте:

In [76]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-LOC', 'B-ORG', 'B-PER', 'I-PER', 'B-MISC', 'I-ORG', 'I-LOC', 'I-MISC']

In [77]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)

0.7717518512048152

### А теперь отдельно для каждого тэга:

In [21]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

       B-LOC      0.589     0.590     0.590      1084
       I-LOC      0.280     0.289     0.284       325
      B-MISC      0.433     0.115     0.182       339
      I-MISC      0.465     0.205     0.284       557
       B-ORG      0.669     0.683     0.676      1400
       I-ORG      0.634     0.753     0.688      1104
       B-PER      0.684     0.747     0.714       735
       I-PER      0.758     0.868     0.809       634

   micro avg      0.626     0.611     0.618      6178
   macro avg      0.564     0.531     0.528      6178
weighted avg      0.608     0.611     0.598      6178



### Посмотрим на наиболее и наименее вероятные переходы модели: 

In [22]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-LOC  -> I-LOC   5.412020
I-LOC  -> I-LOC   5.172153
I-MISC -> I-MISC  5.144828
B-PER  -> I-PER   4.802833
B-MISC -> I-MISC  4.414769
I-PER  -> I-PER   4.343266
B-ORG  -> I-ORG   3.883694
I-ORG  -> I-ORG   3.715221
O      -> O       2.557314
O      -> B-ORG   0.960046
O      -> B-LOC   0.708120
O      -> B-MISC  0.626017
O      -> B-PER   0.279963
B-LOC  -> B-LOC   0.124138
I-LOC  -> O       0.035224
I-PER  -> B-LOC   -0.087076
B-LOC  -> O       -0.102480
I-PER  -> O       -0.187406
B-PER  -> O       -0.227854
B-ORG  -> O       -0.230901

Top unlikely transitions:
B-PER  -> I-ORG   -2.175640
B-LOC  -> I-PER   -2.176378
I-ORG  -> I-LOC   -2.208642
I-ORG  -> I-PER   -2.225142
B-LOC  -> I-ORG   -2.252632
I-ORG  -> B-MISC  -2.253004
I-ORG  -> B-ORG   -2.435161
B-ORG  -> I-PER   -2.480095
I-PER  -> B-ORG   -2.491719
I-MISC -> I-ORG   -2.564156
B-ORG  -> B-MISC  -2.639295
I-ORG  -> I-MISC  -2.799916
B-LOC  -> B-ORG   -2.814176
B-PER  -> B-ORG   -2.846970
B-ORG  -> B-